# A7: Channel Coding & HARQ

This notebook demonstrates basic simulations for channel coding, decoding, and Hybrid ARQ (HARQ) mechanisms, which are core to 5G reliability.

In [ ]:
# Install commpy if not available (used for convolutional codes)
# !pip install commpy

import numpy as np
import matplotlib.pyplot as plt
from commpy.channelcoding import Trellis, conv_encode, viterbi_decode
from numpy.random import rand, randint

## 1. Define a convolutional code

In [ ]:
# Constraint length 3, generator polynomials (7, 5) in octal
trellis = Trellis(memory=np.array([2]), g_matrix=np.array([[7, 5]]))

# Generate random data
data = randint(0, 2, 100)

# Encode
coded = conv_encode(data, trellis)
coded[:20]

## 2. Transmit through BPSK and AWGN

In [ ]:
def awgn(x, snr_db):
    snr = 10 ** (snr_db / 10.0)
    sigma = np.sqrt(1 / (2 * snr))
    noise = sigma * np.random.randn(*x.shape)
    return x + noise

# BPSK modulation
tx_signal = 1 - 2 * coded
rx_signal = awgn(tx_signal, snr_db=3)

# Hard decision demodulation
rx_bits = (rx_signal < 0).astype(int)

## 3. Viterbi Decoding

In [ ]:
# Decode using Viterbi algorithm
decoded = viterbi_decode(rx_bits, trellis, tb_depth=15)

# Compute BER
ber = np.mean(data != decoded[:len(data)])
print(f"Bit Error Rate (BER): {ber:.4f}")

## 4. HARQ with Retransmissions (Chase Combining)

In [ ]:
n_retx = 3
combined_signal = np.zeros_like(tx_signal, dtype=float)

for i in range(n_retx):
    rx = awgn(tx_signal, snr_db=1)  # Low SNR to simulate need for HARQ
    combined_signal += rx

# Average received signal (Chase combining)
rx_combined = combined_signal / n_retx
rx_combined_bits = (rx_combined < 0).astype(int)
decoded_combined = viterbi_decode(rx_combined_bits, trellis, tb_depth=15)

ber_combined = np.mean(data != decoded_combined[:len(data)])
print(f"BER after {n_retx} HARQ retransmissions (Chase Combining): {ber_combined:.4f}")

## 5. Visual Comparison

In [ ]:
snrs = np.arange(0, 6, 1)
ber_noharq = []
ber_harq = []

for snr in snrs:
    # No HARQ
    rx = awgn(tx_signal, snr)
    rx_bits = (rx < 0).astype(int)
    decoded = viterbi_decode(rx_bits, trellis, tb_depth=15)
    ber_noharq.append(np.mean(data != decoded[:len(data)]))
    
    # HARQ
    combined_signal = np.zeros_like(tx_signal, dtype=float)
    for _ in range(n_retx):
        combined_signal += awgn(tx_signal, snr)
    rx_combined = combined_signal / n_retx
    rx_bits_combined = (rx_combined < 0).astype(int)
    decoded = viterbi_decode(rx_bits_combined, trellis, tb_depth=15)
    ber_harq.append(np.mean(data != decoded[:len(data)]))

plt.semilogy(snrs, ber_noharq, label='No HARQ')
plt.semilogy(snrs, ber_harq, label='HARQ (Chase)', linestyle='--')
plt.xlabel('SNR (dB)')
plt.ylabel('BER')
plt.title('BER vs SNR with and without HARQ')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("ber_harq_comparison.png")
plt.show()